# Read the PDF

In [85]:
# libraries for text cleaning
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer

# library to read the PDF
%pip install feedparser
import pdfplumber

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henrytran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/henrytran/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# library to read the PDF
%pip install feedparser
import pdfplumber

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [102]:
# We should use this code
# this is the text extracted from the first 5 pages extracted from the
def raw_data_extraction(input_pdf):
    # take the pdf as an input and return the whole pdf as text
    raw_input=""
    with pdfplumber.open(input_pdf) as pdf:
        for page in pdf.pages:
            raw_input+=(page.extract_text() or "") + "\n"
    return raw_input

def article_extraction(raw_input):
    #all_news = re.sub(r'\([^)]*\)', '', raw_input)
    #sww=set(sw.words())
    article=""
    article_status=False # mark the start of the article
    for row in raw_input.lower().split('\n'):
        if row=='full text':
            article_status=True
            continue
        if article_status:
            if row=='details':
                article_status=False
                continue
            article+=row+" "
        #tokens = re.sub(r"[^a-z0-9]+", " ", row)
        #words = word_tokenize(tokens)
    return article

# temporaryly do not need this
def clean_article(article):
    lemmatizer=WordNetLemmatizer()
    desc = re.sub(r'\([^)]*\)', '', article)
    sent_desc =sent_tokenize(desc)
    sww=set(sw.words())
    abstraction = []
    for sent in sent_desc:
        tokens = re.sub(r"[^a-z0-9]+", " ", sent.lower())
        words = word_tokenize(tokens)
        remaining_words = [word for word in words if word not in sww]
        if remaining_words:
            lemmatised_words=[]
            for word in remaining_words:
                lemmatised_word=lemmatizer.lemmatize(word)
                lemmatised_words.append(lemmatised_word)
            abstraction.append(lemmatised_words)
    final_doc=""
    for sent in abstraction[:-1]:
        for word in sent:
            final_doc+=word+" "
    return final_doc # remove the name of the writer

raw_input1=raw_data_extraction('3pages.pdf')
article1=article_extraction(raw_input1)
doc1=clean_article(article1)
print(raw_input1)

Document 1 of 222
U.S. News: BLS Pick Has Forged Partisan Path ---
Nominee has praised Trump's policies but lacks the
experience of his predecessors
Kiernan, Paul . Kiernan, Paul.
ProQuest document link
FULL TEXT
Conservative economist Erwin John "E.J." Antoni sometimes jokes on social media that the "L" in BLS is silent.
President Trump this week tapped Antoni to run the Bureau of Labor Statistics, the agency whose data and
methodologies he has long criticized, especially when it produces numbers that Trump doesn't like. Antoni recently
proposed suspending the monthly jobs report, one of the most important data releases for the economy and
markets. On Tuesday, a White House official noted that Antoni made the comment before he knew he was going to be
chosen and that his comments don't reflect official BLS policy.
If confirmed by the Senate, Antoni would run a 141-year-old agency staffed by around 2,000 economists, statisticians
and other officials. The BLS has a long record of indepen

# Create the set of fiscal policy related words

In [92]:
# the set of recurring words in these articles related to fiscal policy
fiscal_terms=['tariffs', 'tax cuts', 'budget deficits', 'presidents', 'fiscal policy', 'political campaigns', 'federal budget', 'economic growth', 'interest rates',
                'tax increases', 'gross domestic product-gdp', 'tax rates', 'economists', 'government spending', 'investments', 'voters', 'inflation', 'legislators', 'taxes',
                'federal reserve monetory policy', 'investors', 'presidential elections', 'congressional committees', 'borrowing', 'central banks', 'immigration policy', 
                'international trade', 'economic conditions', 'legislation', 'american dollar', 'social security', 'dow jones averages', 'pandemics', 'treasuries', 'costs',
                'funding', 'manufacturing', 'political leadership', 'clean technology', 'national debt', 'national security', 'stocks', 'exports', 'immigrants',
                'leadership', 'medicare', 'political parties', 'prime ministers', 'scandals', 'tax credits', 'trade policy', 'border security', 'consumer price index',
                'defense spending', 'economic impact', 'economic policy', 'employees', 'federal legislation', 'households', 'immigration', 'international relations', 'low income groups',
                'prices', 'social networks', 'tax legislation', 'trade disputes', 'budgets', 'consumers', 'corporate profits', 'currency', 'deportation', 
                'infrastructure', 'international economic relations', 'nominations', 'nvidia corp', 'polls & surveys', 'provisions', 'recessions', 'securities markets',
                'trade relations', 'bond markets', 'cities', 'consumptions', 'cost control', 'decision making', 'deficit financing', 'economic crisis', 'economic development'
                'enforcement', 'factories', 'federal funding', 'global economy', 'government bonds', 'income taxes', 'international relations-us', 'profits', 'stock exchanges',
                'supply chains', 'tax refunds']

In [ ]:
"""
fiscal_terms={'tax': ['tax cuts','tax increases', 'tax rates', 'tax credits', 'taxes', 'tax legislation', 'income taxes', 'tax refunds'],
                'government_social': ['government spending', 'government bonds', 'defense spending', 'infrastructure','social security', 'medicare', 'low income groups', 'households'],
                'trade': ['tariffs']}
"""

# Match the terms with 

In [99]:
# match the fiscal terms to the document
def count_matching_words(terms_list, document):
    terms_count={}
    for term in terms_list:
        term_count=document.count(term)
        if term_count >0:
            terms_count[term]=term_count
    return terms_count

count_matching_words(fiscal_terms,doc1)

{'fiscal policy': 2, 'inflation': 1, 'consumer price index': 1}

### **This is the code for spaCy**

Can consider using spaCy, because it is more powerful than NLTK, but also harder to use

In [81]:
# use spaCy to match the words
!pip install spacy

In [82]:
import spacy
from spacy.matcher import PhraseMatcher # to match the words in the article with specified terms in the list

In [83]:
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "<frozen runpy>", line 189, in _run_module_as_main
  File "<frozen runpy>", line 148, in _get_module_details
  File "<frozen runpy>", line 112, in _get_module_details
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/__init__.py", line 6, in <module>
    from .errors import setup_default_warnings
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/errors.py", line 3, in <module>
    from .compat import Literal
  File "/opt/anaconda3/lib/python3.12/site-packages/spacy/compat.py", line 4, in <module>
    from thinc.util import copy_array
  File "/opt/anaconda3/lib/python3.12/site-packages/thinc/__init__.py", line 5, in <module>
    from .config import registry
  File "/opt/anaconda3/lib/python3.12/site-packages/thinc/config.py", line 5, in <module>
    from .types import Decorator
  File "/opt/anaconda3/lib/python3.12/site-packages/thinc/types.py", line 27, in <module>
    from .compat import cupy, has_cupy
  File "/opt/anaconda3/li

In [84]:
# load the pretrained general english model (small size) - can considered any pretrained language model for finance or economic
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# create the PhraseMatcher object to load the vocabulary and the fiscal policy terms
article=nlp('') 